In [ ]:
# 01_data_preparation.ipynb

# Импорт необходимых библиотек
import os
import docx
import re
import nltk
import pymorphy2
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Скачать необходимые ресурсы для nltk
nltk.download('punkt')
nltk.download('stopwords')

# Пути к папкам с сырыми и предобработанными данными
raw_data_dir = 'data/raw'
processed_data_dir = 'data/processed'

# Инициализация инструментов для предобработки
morph = pymorphy2.MorphAnalyzer()
stop_words = set(stopwords.words('russian'))

def read_docx(file_path):
    """Функция для чтения .docx файла и извлечения текста"""
    try:
        doc = docx.Document(file_path)
        text = []
        for paragraph in doc.paragraphs:
            text.append(paragraph.text)
        return '\n'.join(text)
    except Exception as e:
        print(f"Ошибка при чтении файла {file_path}: {e}")
        return None

def clean_text(text):
    """Функция для очистки текста"""
    # Приведение текста к нижнему регистру
    text = text.lower()
    # Удаление чисел и специальных символов
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    # Токенизация текста
    words = word_tokenize(text, language='russian')
    # Удаление стоп-слов и лемматизация
    words = [morph.parse(word)[0].normal_form for word in words if word not in stop_words]
    return ' '.join(words)

def process_files(raw_dir, processed_dir):
    """Функция для обработки всех файлов в папке"""
    for file_name in os.listdir(raw_dir):
        if file_name.endswith('.docx'):
            file_path = os.path.join(raw_dir, file_name)
            text = read_docx(file_path)
            if text is not None:
                cleaned_text = clean_text(text)
                processed_file_path = os.path.join(processed_dir, file_name.replace('.docx', '.txt'))
                with open(processed_file_path, 'w', encoding='utf-8') as f:
                    f.write(cleaned_text)
            else:
                print(f"Файл {file_name} пропущен из-за ошибки при чтении.")

# Создание папки для предобработанных данных, если она не существует
os.makedirs(processed_data_dir, exist_ok=True)

# Обработка файлов
process_files(raw_data_dir, processed_data_dir)

print("Предобработка завершена. Файлы сохранены в папку:", processed_data_dir )
